<a href="https://colab.research.google.com/github/jahnavikoneti21/Blindness-Detection/blob/main/Blindness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [12]:
!kaggle competitions download -c aptos2019-blindness-detection

100% 9.51G/9.51G [01:19<00:00, 241MB/s]
100% 9.51G/9.51G [01:19<00:00, 129MB/s]


In [13]:
!unzip \*.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_images/512beaada8f0.png  
  inflating: test_images/51323e9d2070.png  
  inflating: test_images/515675001b9e.png  
  inflating: test_images/515877da2def.png  
  inflating: test_images/5168ddccea88.png  
  inflating: test_images/5172c056a687.png  
  inflating: test_images/5177461f5339.png  
  inflating: test_images/51910bcb980d.png  
  inflating: test_images/51a3e9b6a1c0.png  
  inflating: test_images/51bbd82feec8.png  
  inflating: test_images/51de5fb8eccd.png  
  inflating: test_images/51e88694c81f.png  
  inflating: test_images/51f0bf8f1544.png  
  inflating: test_images/51f57fb32840.png  
  inflating: test_images/51fa42dc1415.png  
  inflating: test_images/5276ab33b7b0.png  
  inflating: test_images/52a353760f72.png  
  inflating: test_images/531937254e73.png  
  inflating: test_images/5325b1c22c61.png  
  inflating: test_images/53959e0a750b.png  
  inflating: test_images/53be22c72713.png  
  inflating: test_images/

In [19]:

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
from PIL import Image
import glob
import os

In [22]:
x_train =pd.read_csv("train.csv")
x_test=  pd.read_csv("test.csv")

In [16]:
x_train.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


import numpy as np
from google.colab import autoviz
df_1105292869412153707 = autoviz.get_df('df_1105292869412153707')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_1105292869412153707, *['diagnosis'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1105292869412153707 = autoviz.get_df('df_1105292869412153707')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_1105292869412153707, *['diagnosis'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1105292869412153707 = autoviz.get_df('df_1105292869412153707')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_1105292869412153707, *['id_code'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1105292869412153707 = autoviz.get_df('df_1105292869412153707')

def violin_plot(df, value_colname, facet_colname, col_width=.3, col_length=3, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  plt.figure(figsize=(col_length, col_width * len(df[facet_colname].unique())))
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(df_1105292869412153707, *['diagnosis', 'id_code'], **{'inner': 'stick'})
chart

In [20]:
def preprocess_image(image_path, img_height, img_width):
    img = Image.open(image_path)
    img = img.resize((img_height, img_width))
    img = np.array(img)
    img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    return img


In [61]:
def load_train_data(train_csv_path,train_images_path , img_height, img_width):
    train_df = pd.read_csv("train.csv")
    train_labels = train_df['diagnosis'].values
    train_images = []

    for image_id in train_df['id_code']:
        image_filename = f"{image_id}.png"  # Assuming images are in JPG format
        image_path = os.path.join(train_images_path, image_filename)
        img = preprocess_image(image_path, img_height, img_width)
        train_images.append(img)
    train_images = np.array(train_images)
    return train_images, train_labels

In [76]:
def load_test_data(test_csv_path, test_images_path, img_height, img_width):
    test_df = pd.read_csv('test.csv')
    test_ids = test_df['id_code'].values
    test_images = []

    for image_id in test_ids:
        image_filename = f"{image_id}.png"  # Assuming images are in JPG format
        image_path = os.path.join(test_images_path, image_filename)
        img = preprocess_image(image_path, img_height, img_width)
        test_images.append(img)

    test_images = np.array(test_images)
    return test_images, test_ids



In [72]:
train_csv_path = "/content/train.csv"
test_csv_path = "/content/test.csv"
train_images_path = "/content/train_images"
test_images_path = "/content/test_images"


In [73]:
img_height, img_width = 128, 128

In [74]:
train_images, train_labels = load_train_data(train_csv_path, train_images_path, img_height, img_width)


In [77]:
test_images, test_ids = load_test_data(test_csv_path, test_images_path, img_height, img_width)

In [98]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [99]:

# Build the CNN model for multi-class classification
num_classes = 5  # Number of severity levels (0 to 4)
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(num_classes, activation='softmax')  # Multi-class classification with softmax activation
])

In [100]:

# Compile the model with categorical crossentropy loss for multi-class classification
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

In [101]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 128)     

In [102]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [106]:
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) // 32,  # Adjust the steps per epoch accordingly
                    epochs=20,  # You can adjust the number of epochs as needed
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stopping])

Epoch 1/20
91/91 [==============================] - 68s 741ms/step - loss: 1.0427 - accuracy: 0.6148 - val_loss: 0.8347 - val_accuracy: 0.7326
Epoch 2/20
91/91 [==============================] - 67s 739ms/step - loss: 0.8808 - accuracy: 0.6955 - val_loss: 0.8055 - val_accuracy: 0.7299
Epoch 3/20
91/91 [==============================] - 69s 762ms/step - loss: 0.8681 - accuracy: 0.7031 - val_loss: 0.8123 - val_accuracy: 0.7285
Epoch 4/20
91/91 [==============================] - 69s 759ms/step - loss: 0.8396 - accuracy: 0.7073 - val_loss: 0.8013 - val_accuracy: 0.7135
Epoch 5/20
91/91 [==============================] - 70s 764ms/step - loss: 0.8394 - accuracy: 0.6969 - val_loss: 0.7995 - val_accuracy: 0.7285
Epoch 6/20
91/91 [==============================] - 68s 746ms/step - loss: 0.8337 - accuracy: 0.7076 - val_loss: 0.8004 - val_accuracy: 0.7353
Epoch 7/20
91/91 [==============================] - 70s 763ms/step - loss: 0.8239 - accuracy: 0.7107 - val_loss: 0.8398 - val_accuracy: 0.7217

In [107]:
predictions = model.predict(test_images)

61/61 [==============================] - 11s 179ms/step


In [109]:
predicted_labels = np.argmax(predictions, axis=1)

In [110]:
#Label 0: No DR or Normal
#Label 1: Mild DR
#Label 2: Moderate DR
#Label 3: Severe DR
for i in range(len(test_ids)):
    print(f"Image ID: {test_ids[i]}, Prediction: {predicted_labels[i]}")

Image ID: 0005cfc8afb6, Prediction: 1
Image ID: 003f0afdcd15, Prediction: 2
Image ID: 006efc72b638, Prediction: 2
Image ID: 00836aaacf06, Prediction: 2
Image ID: 009245722fa4, Prediction: 2
Image ID: 009c019a7309, Prediction: 2
Image ID: 010d915e229a, Prediction: 2
Image ID: 0111b949947e, Prediction: 0
Image ID: 01499815e469, Prediction: 2
Image ID: 0167076e7089, Prediction: 2
Image ID: 01c31b10ab99, Prediction: 2
Image ID: 01c5ba195207, Prediction: 2
Image ID: 01e4d86b3a30, Prediction: 2
Image ID: 020921b796d5, Prediction: 2
Image ID: 020f6983114d, Prediction: 2
Image ID: 021c207614d6, Prediction: 2
Image ID: 0229c0a80d42, Prediction: 1
Image ID: 024d0a225db1, Prediction: 2
Image ID: 0268f4382c67, Prediction: 2
Image ID: 0299d97f31f7, Prediction: 0
Image ID: 03042a663e54, Prediction: 2
Image ID: 030e06ddbb04, Prediction: 2
Image ID: 033cdbbbdfaa, Prediction: 2
Image ID: 03be80919be4, Prediction: 2
Image ID: 03eaa4eef484, Prediction: 2
Image ID: 0434995d0654, Prediction: 2
Image ID: 04

In [114]:

print(f"Image ID: {test_ids[1]}, Prediction: {predicted_labels[1]}")





Image ID: 003f0afdcd15, Prediction: 2
